In [45]:
import numpy as np
from scipy.linalg import eigh, qr, null_space
import matplotlib.pyplot as plt
from scipy.sparse import kron, identity, csr_matrix, lil_matrix, dok_matrix, coo_matrix, issparse
from scipy.sparse.linalg import eigsh, eigs
from scipy.special import factorial, comb
from scipy.optimize import curve_fit
from qutip import Qobj, ptrace, entropy_vn, qeye, tensor
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from joblib import Parallel, delayed
from itertools import combinations
#from quspin.basis import spin_basis_1d, spin_basis_general
#import tenpy as tp

In [46]:
# spin-1/2 basis states + spin operators

# basis states for spin-1/2 system

# |up> state
ket_p = csr_matrix([[1], [0]])

# |down> state
ket_m = csr_matrix([[0], [1]])

# Spin-1/2 operators as sparse matrices
sx = csr_matrix([[0, 1], [1, 0]], dtype=np.complex128)
sy = csr_matrix([[0, -1j], [1j, 0]], dtype=np.complex128)
sz = csr_matrix([[1, 0], [0, -1]], dtype=np.complex128)

I = identity(2, format='csr', dtype=np.complex128)

# s_+ operator
sp = (1/2) * (sx + 1j * sy)

# s_- operator
sm = (1/2) * (sx - 1j * sy)

# P_0 operator
P_0 = (1/2) * (I - sz)
P_0p = (1/2) * (I + sz)

# --- Vacuum state |omega> = |0>^{⊗L} ---
def omega(L):
    state = ket_m
    for _ in range(L-1):
        state = kron(state, ket_m, format='csr')
    return state

def omega_p(L):
    state = ket_p
    for _ in range(L-1):
        state = kron(state, ket_p, format='csr')
    return state

# --- Q^+ operator ---
def Q_plus(L, bc):
    Qp = csr_matrix((2**L, 2**L), dtype=complex)
    for i in range(2, L):
        phase = np.exp(1j * np.pi * i)
        qplus =1
        for j in range(1, L+1):
            if j == i:
                qplus = kron(qplus, sp, format='csr')
            elif j == i-1 or j == i+1:
                qplus = kron(qplus, P_0, format='csr')
            else:
                qplus = kron(qplus, I, format='csr')
        Qp += phase * qplus

    if bc == 'pbc':
        # Add the wraparound terms for PBC

        qplus1 = 1
        for j in range(1, L+1):
            phase = np.exp(1j * np.pi * j)
            if j == 1:
                qplus1 = kron(qplus1, sp, format='csr')
            elif j == 2 or j == L:
                qplus1 = kron(qplus1, P_0, format='csr')
            else:
                qplus1 = kron(qplus1, I, format='csr')
        Qp += phase * qplus1

        qplus2 = 1
        for j in range(1, L+1):
            phase = np.exp(1j * np.pi * j)
            if j == 1 or j == L-1:
                qplus2 = kron(qplus2, P_0, format='csr')
            elif j == L:
                qplus2 = kron(qplus2, sp, format='csr')
            else:
                qplus2 = kron(qplus2, I, format='csr')
        Qp += phase * qplus2
    return Qp

# --- Q^+ operator ---
def Q_plus_p(L, bc):
    Qp = csr_matrix((2**L, 2**L), dtype=complex)
    for i in range(2, L):
        phase = np.exp(1j * np.pi * i)
        qplus =1
        for j in range(1, L+1):
            if j == i:
                qplus = kron(qplus, sm, format='csr')
            elif j == i-1 or j == i+1:
                qplus = kron(qplus, P_0p, format='csr')
            else:
                qplus = kron(qplus, I, format='csr')
        Qp += phase * qplus

    if bc == 'pbc':
        # Add the wraparound terms for PBC

        qplus1 = 1
        for j in range(1, L+1):
            phase = np.exp(1j * np.pi * j)
            if j == 1:
                qplus1 = kron(qplus1, sm, format='csr')
            elif j == 2 or j == L:
                qplus1 = kron(qplus1, P_0p, format='csr')
            else:
                qplus1 = kron(qplus1, I, format='csr')
        Qp += phase * qplus1

        qplus2 = 1
        for j in range(1, L+1):
            phase = np.exp(1j * np.pi * j)
            if j == 1 or j == L-1:
                qplus2 = kron(qplus2, P_0p, format='csr')
            elif j == L:
                qplus2 = kron(qplus2, sm, format='csr')
            else:
                qplus2 = kron(qplus2, I, format='csr')
        Qp += phase * qplus2
    return Qp


# --- Tower state |n> ---
def tower_state(n, L, bc):
    if bc == 'obc':
        norm = factorial(n) * np.sqrt(comb(L-n-1, n))
    elif bc == 'pbc':
        norm = factorial(n) * np.sqrt((L/n) * comb(L-n-1, n-1))
    else:
        raise ValueError("Boundary condition must be 'obc' or 'pbc'.")

    Qp = Q_plus(L, bc)
    psi = omega(L)
    for _ in range(n):
        psi = Qp.dot(psi)
    return psi / norm

# --- Tower state |n> ---
def tower_state_p(n, L, bc):
    if bc == 'obc':
        norm = factorial(n) * np.sqrt(comb(L-n-1, n))
    elif bc == 'pbc':
        norm = factorial(n) * np.sqrt((L/n) * comb(L-n-1, n-1))
    else:
        raise ValueError("Boundary condition must be 'obc' or 'pbc'.")

    Qp = Q_plus_p(L, bc)
    psi = omega_p(L)
    for _ in range(n):
        psi = Qp.dot(psi)
    return psi / norm

def hamiltonian_dwall(L, lam, delta, J, bc):
    """
    Constructs the domain wall Hamiltonian using sparse matrices.
    H = sum_i [λ(σx_i - σz_{i-1} σx_i σz_{i+1}) + δ σz_i + J σz_i σz_{i+1}]
    
    Args:
        L: number of sites
        lam: λ parameter (coupling for σx and three-body terms)
        delta: δ parameter (magnetic field strength)
        J: Ising coupling strength
        bc: boundary conditions ('obc' or 'pbc')
    
    Returns:
        H: sparse Hamiltonian matrix
    """
    H = csr_matrix((2**L, 2**L), dtype=complex)
    
    # Hλ term: λ σx_i for each site
    for i in range(2,L):
        hx1_term = 1
        hx2_term = 1
        for j in range(1,L+1):
            if j == i:
                hx1_term = kron(hx1_term, sx, format='csr')
                hx2_term = kron(hx2_term, sx, format='csr')
            elif j == i-1 or j == i+1:
                hx1_term = kron(hx1_term, I, format='csr')
                hx2_term = kron(hx2_term, sz, format='csr')
            else:
                hx1_term = kron(hx1_term, I, format='csr')
                hx2_term = kron(hx2_term, I, format='csr')
        H += lam * (hx1_term - hx2_term)

    # Hz term: δ σz_i for each site
    for i in range(2,L):
        hz_term = 1
        for j in range(1,L+1):
            if j == i:
                hz_term = kron(hz_term, sz, format='csr')
            else:
                hz_term = kron(hz_term, I, format='csr')
        H += delta * hz_term
    
    # Hzz term: J σz_i σz_{i+1}
    for i in range(2,L):
        hzz_term = 1
        for j in range(1,L+1):
            if j == i or j == i+1:
                hzz_term = kron(hzz_term, sz, format='csr')
            else:
                hzz_term = kron(hzz_term, I, format='csr')
        H += J * hzz_term
    
    # For PBC, add the wraparound Ising term
    if bc == 'pbc':
        hzz_wrap = 1
        for j in range(L):
            if j == 0 or j == L-1:
                hzz_wrap = kron(hzz_wrap, sz, format='csr')
            else:
                hzz_wrap = kron(hzz_wrap, I, format='csr')
        H += J * hzz_wrap

    if bc == 'pbc':
    # Add the wraparound terms for PBC

        # Hλ term
        hx1_term = 1
        for j in range(1, L+1):
            if j == 1 or j == L:
                hx1_term = kron(hx1_term, sx, format='csr')
            else:
                hx1_term = kron(hx1_term, I, format='csr')
        hx2_term1 = 1
        hx2_term2 = 1
        for j in range(1, L+1):
            if j == 1:
                hx2_term1 = kron(hx2_term1, sx, format='csr')
                hx2_term2 = kron(hx2_term2, sz, format='csr')
            elif j == 2:
                hx2_term1 = kron(hx2_term1, sz, format='csr')
                hx2_term2 = kron(hx2_term2, I, format='csr')
            elif j == L-1:
                hx2_term1 = kron(hx2_term1, I, format='csr')
                hx2_term2 = kron(hx2_term2, sz, format='csr')
            elif j == L:
                hx2_term1 = kron(hx2_term1, sz, format='csr')
                hx2_term2 = kron(hx2_term2, sx, format='csr')
            else:
                hx2_term1 = kron(hx2_term1, I, format='csr')
                hx2_term2 = kron(hx2_term2, I, format='csr')
        H += lam * (hx1_term - hx2_term1 - hx2_term2)

        #Hz term
        hz_term = 1
        for j in range(1, L+1):
            if j == 1 or j == L:
                hz_term = kron(hz_term, sz, format='csr')
            else:
                hz_term = kron(hz_term, I, format='csr')
        H += delta * hz_term

        # Hzz term
        hzz_term1 = 1
        hzz_term2 = 1
        for j in range(1, L+1):
            if j == 1:
                hzz_term1 = kron(hzz_term1, sz, format='csr')
                hzz_term2 = kron(hzz_term2, sz, format='csr')
            elif j == 2:
                hzz_term1 = kron(hzz_term1, sz, format='csr')
                hzz_term2 = kron(hzz_term2, I, format='csr')
            elif j == L:
                hzz_term1 = kron(hzz_term1, I, format='csr')
                hzz_term2 = kron(hzz_term2, sz, format='csr')
            else:
                hzz_term1 = kron(hzz_term1, I, format='csr')
                hzz_term2 = kron(hzz_term2, I, format='csr')
        H += J * (hzz_term1 + hzz_term2)
    
    return H

In [47]:
# functions

def innermost_adjacent_indices(L, block_size):
    """
    Returns the indices of the innermost adjacent block of given size.
    For even L, the block is centered in the middle.
    """
    start = (L - block_size) // 2
    return list(range(start, start + block_size))

def all_adjacent_indices(L, block_size):
    """
    Returns a list of all possible adjacent blocks of given size.
    Each block is represented as a list of indices.
    """
    return [list(range(start, start + block_size)) for start in range(L - block_size + 1)]

def ptrace_sparse(dm_sparse, keep, dims):
    """
    Compute the partial trace over arbitrary subsystems using sparse matrix operations.

    Args:
        dm_sparse (scipy.sparse matrix): Full density matrix of shape (D, D), where D = product(dims)
        keep (list of int): Subsystems to keep (indices, 0-indexed)
        dims (list of int): List of subsystem dimensions, e.g., [2]*n for n qubits

    Returns:
        scipy.sparse.csr_matrix: Reduced density matrix over kept subsystems
    """
    if not issparse(dm_sparse):
        raise ValueError("dm_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if dm_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])
    # Prepare output
    data = []
    row_idx = []
    col_idx = []

    # Precompute bit masks
    #def idx_to_bits(idx):
    #    return np.array(list(np.binary_repr(idx, width=n))).astype(int)

    def idx_to_subsys(idx, dims):
    #Convert flat index to tuple of subsystem indices for arbitrary dims.
        subsys = []
        for d in reversed(dims):
            subsys.append(idx % d)
            idx //= d
        return np.array(subsys[::-1])

    

    dm_sparse = dm_sparse.tocoo()

    for i, j, val in tqdm(zip(dm_sparse.row, dm_sparse.col, dm_sparse.data)):
        bi = idx_to_subsys(i, dims)
        bj = idx_to_subsys(j, dims)

        if np.all(bi[trace] == bj[trace]):
            i_red = 0
            j_red = 0
            for k, pos in enumerate(keep):
                i_red = i_red * dims[pos] + bi[pos]
                j_red = j_red * dims[pos] + bj[pos]

            data.append(val)
            row_idx.append(i_red)
            col_idx.append(j_red)

    
    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()

def ptrace_sparse_parallel(dm_sparse, keep, dims, n_jobs=-1): # njobs to be removed if not using joblib
    """
    Compute the partial trace over arbitrary subsystems using sparse matrix operations.
    Parallelized over nonzero elements.
    """
    if not issparse(dm_sparse):
        raise ValueError("dm_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if dm_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])


    def idx_to_subsys(idx, dims):
    #Convert flat index to tuple of subsystem indices for arbitrary dims.
        subsys = []
        for d in reversed(dims):
            subsys.append(idx % d)
            idx //= d
        return np.array(subsys[::-1])

    
    dm_sparse = dm_sparse.tocoo()

    def process_entry(i,j,val):
        bi = idx_to_subsys(i, dims)
        bj = idx_to_subsys(j, dims)

        if np.all(bi[trace] == bj[trace]):
            i_red = 0
            j_red = 0
            for k, pos in enumerate(keep):
                i_red = i_red * dims[pos] + bi[pos]
                j_red = j_red * dims[pos] + bj[pos]
            return (val, i_red, j_red)
        else:
            return None
        
    results = Parallel(n_jobs=n_jobs, prefer="processes")(
        delayed(process_entry)(i, j, val)
        for i, j, val in tqdm(zip(dm_sparse.row, dm_sparse.col, dm_sparse.data))
    )
    results = [r for r in results if r is not None]

    '''entries = zip(psi_sparse.row, psi_sparse.col, psi_sparse.data)
    results = []
    with ThreadPoolExecutor() as executor:
        for res in executor.map(process_entry, entries):
            if res is not None:
                results.append(res)'''
    
    if results:
        data, row_idx, col_idx = zip(*results)
    else:
        data, row_idx, col_idx = [], [], []

    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()

def ee_sparse(dm_sparse, L):
    """
    Computes the entanglement entropy of a state using sparse matrices in parallel.
    The state is assumed to be a vector in the Hilbert space of L qubits.
    """
    rhoA = ptrace_sparse(dm_sparse, list(range(L // 2)), [2] * L)
    eigvals = np.linalg.eigvalsh(rhoA.toarray())
    return -np.sum(eigvals * np.log(eigvals + 1e-12)).real  # Add small value to avoid log(0)

def ee_sparse_parallel(dm_sparse, L, n_jobs=-1):
    """
    Computes the entanglement entropy of a state using sparse matrices in parallel.
    The state is assumed to be a vector in the Hilbert space of L qubits.
    """
    rhoA = ptrace_sparse_parallel(dm_sparse, list(range(L // 2)), [2] * L, n_jobs=n_jobs)
    eigvals = np.linalg.eigvalsh(rhoA.toarray())
    return -np.sum(eigvals * np.log(eigvals + 1e-12)).real  # Add small value to avoid log(0)

def rdm_qutip(state, L, keep_qubits):
    rho = np.outer(state, state.conj())
    rho_qobj = Qobj(rho, dims=[[2] * L, [2] * L])
    rdm = ptrace(rho_qobj, keep_qubits)
    rdm_mat = rdm.full()
    eigvals = np.linalg.eigvalsh(rdm_mat)
    min_eigval = np.min(eigvals)
    # Rank: count nonzero eigenvalues (with tolerance)
    rank = np.sum(eigvals > 1e-12)
    return rdm, min_eigval, rank

def ee_qutip(state, L):
    rho = np.outer(state, state.conj())
    rho_qobj = Qobj(rho, dims=[[2] * L, [2] * L])
    rhoA = ptrace(rho_qobj, list(range(L//2)))
    return entropy_vn(rhoA)

In [48]:
L = 12 # number of sites -  it has to be even

innermost_2 = innermost_adjacent_indices(L, 2)
innermost_3 = innermost_adjacent_indices(L, 3)
innermost_4 = innermost_adjacent_indices(L, 4)

adjacent_2 = all_adjacent_indices(L, 2)
adjacent_3 = all_adjacent_indices(L, 3)
adjacent_4 = all_adjacent_indices(L, 4)

print("All adjacent 2-site blocks:", adjacent_2)
print("All adjacent 3-site blocks:", adjacent_3)
print("All adjacent 4-site blocks:", adjacent_4)

All adjacent 2-site blocks: [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11]]
All adjacent 3-site blocks: [[0, 1, 2], [1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9, 10], [9, 10, 11]]
All adjacent 4-site blocks: [[0, 1, 2, 3], [1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 5, 6], [4, 5, 6, 7], [5, 6, 7, 8], [6, 7, 8, 9], [7, 8, 9, 10], [8, 9, 10, 11]]


In [49]:
# DOMAIN WALL - PRB 024306 - qutip version

H = hamiltonian_dwall(L, lam=1.0, delta=0.1, J=1.0, bc='pbc')
ground_state = eigsh(H, k=1, which='SA', return_eigenvectors=True)[1][:, 0]  # Get the ground state vector
print(f"ground state dimension for L={L}: {ground_state.shape}")

# Print number of zero components (with tolerance 1e-12)
#print(np.count_nonzero(scar_state))
num_zeros = np.sum(np.abs(ground_state) > 1e-16)
print(f"Number of zero components in ground_state (tol=1e-12): {num_zeros}")

scar_sparse = csr_matrix(ground_state.reshape(-1, 1))  # Convert to sparse column vector
density_matrix_sparse = scar_sparse @ scar_sparse.getH()  # Outer product to form density matrix
density_matrix_qobj = Qobj(density_matrix_sparse, dims=[[2]*L, [2]*L])  # Convert to Qobj for qutip
print("Number of zero elements of dm (tol=1e-12):", np.sum(np.abs(density_matrix_sparse.data) > 1e-16))    #Trace out qubits using qutip partial trace

# Calculate RDMs for all possible adjacent 2, 3, 4 site blocks
for block_size, all_blocks in zip([2, 3, 4], [adjacent_2, adjacent_3, adjacent_4]):
    print(f"\nAll possible RDMs for block size {block_size}:")
    for block_indices in tqdm(all_blocks):
        rdm = ptrace(density_matrix_qobj, block_indices)
        # Find the minimum eigenvalue of the traced-out density matrix
        eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.full())
        rank = np.linalg.matrix_rank(rdm.full())
        min_eigenvalue = np.min(eigenvalues_traced)
        print(f"Block {block_indices}: min eigenvalue = {min_eigenvalue}, rank = {rank}")
'''density_matrix_qobj = Qobj(density_matrix_sparse, dims=[[2]*L, [2]*L])  # Convert to Qobj for qutip
rdm = ptrace(density_matrix_qobj, adjacent_4[0])
# Find the minimum eigenvalue of the traced-out density matrix
eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.full())
rank = np.linalg.matrix_rank(rdm.full())
min_eigenvalue = np.min(eigenvalues_traced)
print(f"min eigenvalue = {min_eigenvalue}, rank = {rank}")'''

ground state dimension for L=12: (4096,)
Number of zero components in ground_state (tol=1e-12): 4041
Number of zero elements of dm (tol=1e-12): 16253564

All possible RDMs for block size 2:


Block [0, 1]: min eigenvalue = 0.04209618104968535, rank = 4


Block [1, 2]: min eigenvalue = 0.013356765526957212, rank = 4
Block [2, 3]: min eigenvalue = 0.021318675383178155, rank = 4


 27%|██▋       | 3/11 [00:02<00:06,  1.23it/s]

Block [3, 4]: min eigenvalue = 0.02038018160726073, rank = 4


...

Block [6, 7, 8, 9]: min eigenvalue = 3.1897214242404023e-06, rank = 16


Block [7, 8, 9, 10]: min eigenvalue = 1.3117380787425933e-06, rank = 16


100%|██████████| 9/9 [00:11<00:00,  1.25s/it]

Block [8, 9, 10, 11]: min eigenvalue = 5.8747798748656115e-06, rank = 16


'density_matrix_qobj = Qobj(density_matrix_sparse, dims=[[2]*L, [2]*L])  # Convert to Qobj for qutip\nrdm = ptrace(density_matrix_qobj, adjacent_4[0])\n# Find the minimum eigenvalue of the traced-out density matrix\neigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.full())\nrank = np.linalg.matrix_rank(rdm.full())\nmin_eigenvalue = np.min(eigenvalues_traced)\nprint(f"min eigenvalue = {min_eigenvalue}, rank = {rank}")'

In [50]:
# DOMAIN WALL - PRB 024306

scar_state = tower_state_p(int((L/2)/2 - 1), L, 'pbc')  # with boundary conditions
#dimer_state = dimer_state.flatten()  # Reshape to column vector
print(f"scar state dimension for L={L}: {scar_state.shape}")

# Print number of zero components (with tolerance 1e-12)
#print(np.count_nonzero(scar_state))
num_zeros = np.sum(np.abs(scar_state) > 1e-16)
print(f"Number of zero components in scar_state (tol=1e-12): {num_zeros}")

scar_sparse = csr_matrix(scar_state.reshape(-1, 1))  # Convert to sparse column vector
density_matrix_sparse = scar_sparse @ scar_sparse.getH()  # Outer product to form density matrix
print("Number of zero elements of dm (tol=1e-12):", np.sum(np.abs(density_matrix_sparse.data) > 1e-16))    #Trace out qubits using qutip partial trace

# Calculate RDMs for all possible adjacent 2, 3, 4 site blocks
for block_size, all_blocks in zip([2, 3, 4], [adjacent_2, adjacent_3, adjacent_4]):
    print(f"\nAll possible RDMs for block size {block_size}:")
    for block_indices in tqdm(all_blocks):
        rdm = ptrace_sparse_parallel(density_matrix_sparse, block_indices, [2]*L, n_jobs=-1) # Use the custom ptrace_sparse function
        # Find the minimum eigenvalue of the traced-out density matrix
        eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.toarray())
        rank = np.linalg.matrix_rank(rdm.toarray())
        min_eigenvalue = np.min(eigenvalues_traced)
        print(f"Block {block_indices}: min eigenvalue = {min_eigenvalue}, rank = {rank}")
'''rdm = ptrace_sparse_parallel(density_matrix_sparse, adjacent_4[0], [2]*L, n_jobs=-1) # Use the custom ptrace_sparse function
# Find the minimum eigenvalue of the traced-out density matrix
eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.toarray())
rank = np.linalg.matrix_rank(rdm.toarray())
min_eigenvalue = np.min(eigenvalues_traced)
print(f"min eigenvalue = {min_eigenvalue}, rank = {rank}")'''

scar state dimension for L=12: (4096, 1)
Number of zero components in scar_state (tol=1e-12): 54
Number of zero elements of dm (tol=1e-12): 2916

All possible RDMs for block size 2:








2916it [00:01, 2740.15it/s]


Block [0, 1]: min eigenvalue = 0.0, rank = 3






2916it [00:00, 6435.91it/s]


Block [1, 2]: min eigenvalue = 0.0, rank = 3


...




2916it [00:00, 11239.96it/s]


Block [7, 8, 9, 10]: min eigenvalue = -7.307049483950634e-18, rank = 5






2916it [00:00, 8057.08it/s]
100%|██████████| 9/9 [00:05<00:00,  1.58it/s]

Block [8, 9, 10, 11]: min eigenvalue = -7.307049483950634e-18, rank = 5


'rdm = ptrace_sparse_parallel(density_matrix_sparse, adjacent_4[0], [2]*L, n_jobs=-1) # Use the custom ptrace_sparse function\n# Find the minimum eigenvalue of the traced-out density matrix\neigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.toarray())\nrank = np.linalg.matrix_rank(rdm.toarray())\nmin_eigenvalue = np.min(eigenvalues_traced)\nprint(f"min eigenvalue = {min_eigenvalue}, rank = {rank}")'

In [51]:
# DOMAIN WALL - PRB 024306 - qutip version

scar_state = tower_state_p(int((L/2)/2 - 1), L, 'pbc')  # with boundary conditions
#dimer_state = dimer_state.flatten()  # Reshape to column vector
print(f"scar state dimension for L={L}: {scar_state.shape}")

# Print number of zero components (with tolerance 1e-12)
#print(np.count_nonzero(scar_state))
num_zeros = np.sum(np.abs(scar_state) > 1e-16)
print(f"Number of zero components in scar_state (tol=1e-12): {num_zeros}")

scar_sparse = csr_matrix(scar_state.reshape(-1, 1))  # Convert to sparse column vector
density_matrix_sparse = scar_sparse @ scar_sparse.getH()  # Outer product to form density matrix
density_matrix_qobj = Qobj(density_matrix_sparse, dims=[[2]*L, [2]*L])  # Convert to Qobj for qutip
print("Number of zero elements of dm (tol=1e-12):", np.sum(np.abs(density_matrix_sparse.data) > 1e-16))    #Trace out qubits using qutip partial trace

# Calculate RDMs for all possible adjacent 2, 3, 4 site blocks
for block_size, all_blocks in zip([2, 3, 4], [adjacent_2, adjacent_3, adjacent_4]):
    print(f"\nAll possible RDMs for block size {block_size}:")
    for block_indices in tqdm(all_blocks):
        rdm = ptrace(density_matrix_qobj, block_indices)
        # Find the minimum eigenvalue of the traced-out density matrix
        eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.full())
        rank = np.linalg.matrix_rank(rdm.full())
        min_eigenvalue = np.min(eigenvalues_traced)
        print(f"Block {block_indices}: min eigenvalue = {min_eigenvalue}, rank = {rank}")

'''rdm = ptrace(density_matrix_qobj, adjacent_4[0])
# Find the minimum eigenvalue of the traced-out density matrix
eigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.full())
rank = np.linalg.matrix_rank(rdm.full())
min_eigenvalue = np.min(eigenvalues_traced)
print(f"min eigenvalue = {min_eigenvalue}, rank = {rank}")'''

scar state dimension for L=12: (4096, 1)
Number of zero components in scar_state (tol=1e-12): 54
Number of zero elements of dm (tol=1e-12): 2916

All possible RDMs for block size 2:


100%|██████████| 11/11 [00:00<?, ?it/s]


Block [0, 1]: min eigenvalue = 0.0, rank = 3
Block [1, 2]: min eigenvalue = 0.0, rank = 3
Block [2, 3]: min eigenvalue = 0.0, rank = 3
Block [3, 4]: min eigenvalue = 0.0, rank = 3
Block [4, 5]: min eigenvalue = 0.0, rank = 3
Block [5, 6]: min eigenvalue = 0.0, rank = 3
Block [6, 7]: min eigenvalue = 0.0, rank = 3
Block [7, 8]: min eigenvalue = 0.0, rank = 3
Block [8, 9]: min eigenvalue = 0.0, rank = 3
Block [9, 10]: min eigenvalue = 0.0, rank = 3
Block [10, 11]: min eigenvalue = 0.0, rank = 3

All possible RDMs for block size 3:


100%|██████████| 10/10 [00:00<00:00, 2838.02it/s]


Block [0, 1, 2]: min eigenvalue = 0.0, rank = 5
Block [1, 2, 3]: min eigenvalue = 0.0, rank = 5
Block [2, 3, 4]: min eigenvalue = 0.0, rank = 5
Block [3, 4, 5]: min eigenvalue = 0.0, rank = 5
Block [4, 5, 6]: min eigenvalue = 0.0, rank = 5
Block [5, 6, 7]: min eigenvalue = 0.0, rank = 5
Block [6, 7, 8]: min eigenvalue = 0.0, rank = 5
Block [7, 8, 9]: min eigenvalue = 0.0, rank = 5
Block [8, 9, 10]: min eigenvalue = 0.0, rank = 5
Block [9, 10, 11]: min eigenvalue = 0.0, rank = 5

All possible RDMs for block size 4:


100%|██████████| 9/9 [00:00<?, ?it/s]

Block [0, 1, 2, 3]: min eigenvalue = -7.307049483950634e-18, rank = 5
Block [1, 2, 3, 4]: min eigenvalue = -7.307049483950634e-18, rank = 5
Block [2, 3, 4, 5]: min eigenvalue = -7.307049483950634e-18, rank = 5
Block [3, 4, 5, 6]: min eigenvalue = -7.307049483950634e-18, rank = 5
Block [4, 5, 6, 7]: min eigenvalue = -7.307049483950634e-18, rank = 5
Block [5, 6, 7, 8]: min eigenvalue = -7.307049483950634e-18, rank = 5
Block [6, 7, 8, 9]: min eigenvalue = -7.307049483950634e-18, rank = 5
Block [7, 8, 9, 10]: min eigenvalue = -7.307049483950634e-18, rank = 5
Block [8, 9, 10, 11]: min eigenvalue = -7.307049483950634e-18, rank = 5


'rdm = ptrace(density_matrix_qobj, adjacent_4[0])\n# Find the minimum eigenvalue of the traced-out density matrix\neigenvalues_traced, eigenvectors_traced = np.linalg.eigh(rdm.full())\nrank = np.linalg.matrix_rank(rdm.full())\nmin_eigenvalue = np.min(eigenvalues_traced)\nprint(f"min eigenvalue = {min_eigenvalue}, rank = {rank}")'

In [52]:
# DOMAIN WALL - PRB 024306

'''
# L is your system size
# Nup = L//2 for Sz=0 sector (number of up spins)
# kblock=0 for momentum k=0 (T=1 eigenvalue)
sym_basis = spin_basis_1d(L, Nup=L//2, kblock=0)
print("Basis size:", sym_basis.Ns)

# get symmetry basis states as integers
proj_states = sym_basis.get_proj(np.arange(sym_basis.Ns))

# project dimer_state onto the symmetry sector
xy_proj = scar_state[proj_states]

# normalize if desired
xy_proj = xy_proj / np.linalg.norm(xy_proj)


# xy_proj is now the state in the (Sz=0, T=1) sector basis
'''


# dimer ee for single L and Ltar dependence
xy_scar_ee = ee_sparse_parallel(density_matrix_sparse, L, n_jobs=-1)
print(f"xy-scar entanglement entropy for L={L}: {xy_scar_ee/np.log(2)}")

#xy_scar_ee_tar = [ee_sparse_parallel(scar_state, Lt, n_jobs=-1) for Lt in tqdm(Ltar)]

#plt.figure(figsize=(6,4))
#plt.plot(Ltar, xy_scar_ee_tar, marker='o')
#plt.xlabel('L')
#plt.ylabel('Entanglement Entropy')
#plt.title('Dimer Entanglement Entropy vs L')
#plt.grid(True)
#plt.show()

2916it [00:00, 8942.61it/s]


xy-scar entanglement entropy for L=12: 1.5102089111093597
